In [2]:
import pandas as pd
from datasets import Dataset, concatenate_datasets

csv_path = './multitask_training/data_cleaned_manual_combined.csv'
model_name = 

df = pd.read_csv(csv_path)

labels_gs = df['Global Subject'].unique()
labels_qi = df['Question Intent'].unique()

id2label_gs = {i: label for i, label in enumerate(labels_gs)}
label2id_gs = {label: i for i, label in enumerate(labels_gs)}

id2label_qi = {i: label for i, label in enumerate(labels_qi)}
label2id_qi = {label: i for i, label in enumerate(labels_qi)}

In [3]:
from transformers import AutoTokenizer, AutoModel
import numpy as np

# https://huggingface.co/docs/transformers/v4.46.2/en/model_doc/auto#transformers.AutoTokenizer.from_pretrained
# https://huggingface.co/docs/transformers/v4.46.2/en/model_doc/bert#transformers.BertTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


def preprocess_data(example):
    question = example['Question']

    # https://huggingface.co/docs/transformers/v4.46.2/en/main_classes/tokenizer#transformers.PreTrainedTokenizer.__call__
    encodings = tokenizer(question, padding="max_length", truncation=True, max_length=128)
    label_gs = label2id_gs[example['Global Subject']]
    label_qi = label2id_qi[example['Question Intent']]

    encodings.update({'labels': label_gs, 'labels_sub': label_qi})

    return encodings

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
from datasets import Dataset

# Assuming df is your DataFrame and it has a column 'Global Subject' for stratification
df = pd.read_csv(csv_path)

# Create the stratified split
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df['Global Subject']):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

# Reset index of strat_test_set
strat_test_set = strat_test_set.reset_index(drop=True)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(strat_train_set)
test_dataset = Dataset.from_pandas(strat_test_set)

# Tokenize the datasets
train_dataset = train_dataset.map(preprocess_data, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(preprocess_data, remove_columns=test_dataset.column_names)

train_dataset.set_format("torch")
test_dataset.set_format("torch")

Map:   0%|          | 0/683 [00:00<?, ? examples/s]

Map:   0%|          | 0/171 [00:00<?, ? examples/s]

In [ ]:
model = AutoModel.from_pretrained()